In [1]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation    
sns.set(color_codes=True)

pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Load the dataset
data_path = '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-1/adsp_ml_ops/data/athletes.csv'
data = pd.read_csv(data_path)

In [3]:
data.head(3)

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220.0,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Raw data

## Data Cleaning

In [4]:
## Remove null values of relevant columns
relevant_columns = ['region', 'age', 'weight', 'height', 'howlong', 'gender', 'eat', 'train', 
                    'background', 'experience', 'schedule', 'deadlift', 'candj', 'snatch', 'backsq']
data = data.dropna(subset=relevant_columns)

In [5]:
## Remove irrelevant columns
irrelevant_columns = ['affiliate', 'team', 'name', 'athlete_id', 'fran', 'helen', 'grace',
                      'filthy50', 'fgonebad', 'run400', 'run5k', 'pullups', 'train']
data = data.drop(columns=irrelevant_columns)

In [8]:
data.head(5)

,region,gender,age,height,weight,candj,snatch,deadlift,backsq,eat,background,experience,schedule,howlong
6,South Central,Male,21.0,72.0,175.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amou...,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 3+ times a wee...,2-4 years|
13,Central East,Male,43.0,71.0,185.0,0.0,0.0,0.0,0.0,I eat quality foods but don't measure the amount|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I usually only do 1 workout a day|I typically ...,2-4 years|
21,Southern California,Male,30.0,71.0,200.0,235.0,175.0,385.0,315.0,I eat whatever is convenient|,I played youth or high school level sports|I p...,I began CrossFit by trying it alone (without a...,I do multiple workouts in a day 1x a week|I ty...,1-2 years|
22,Africa,Male,28.0,70.0,176.0,187.0,134.0,335.0,254.0,I eat 1-3 full cheat meals per week|,I have no athletic background besides CrossFit|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 1x a week|,2-4 years|
27,North East,Male,35.0,68.0,225.0,285.0,205.0,440.0,405.0,I eat quality foods but don't measure the amount|,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I typically rest 4 or more days per month|,2-4 years|


In [11]:
# Clean survey data
decline_dict = {'Decline to answer|': np.nan}
data = data.replace(decline_dict)
data = data.dropna(subset=['background', 'experience', 'schedule', 'howlong', 'eat'])


### Determine the highest variability in columns to calculate total_lift

In [6]:
#Calculate standard deviation
numeric_columns = ['age', 'weight', 'height', 'howlong', 'deadlift', 'candj', 'snatch', 'backsq']

data_v1_std_devs = data[numeric_columns].std()

#Sort columns by their standard devation
sorted_columns_v1 = data_v1_std_devs.sort_values(ascending=False).index.tolist()

/var/folders/dn/9qflv0c57rl7ynhwt9tnqmrr0000gn/T/ipykernel_4442/2011793858.py:4: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_v1_std_devs = data[numeric_columns].std()


In [7]:
print(sorted_columns_v1)

['deadlift', 'height', 'backsq', 'snatch', 'candj', 'weight', 'age']


### The highest variability to calculate total_lift is 'deadlift', 'backsq', 'candj', 'snatch'

In [9]:
data['total_lift'] = data['deadlift'] + data['candj'] + data['snatch'] + data['backsq']


In [10]:
data.dtypes

region         object
gender         object
age           float64
height        float64
weight        float64
candj         float64
snatch        float64
deadlift      float64
backsq        float64
eat            object
background     object
experience     object
schedule       object
howlong        object
total_lift    float64
dtype: object

In [11]:
data.shape

(32172, 15)

In [12]:
data.count()

region        32172
gender        32172
age           32172
height        32172
weight        32172
candj         32172
snatch        32172
deadlift      32172
backsq        32172
eat           32172
background    32172
experience    32172
schedule      32172
howlong       32172
total_lift    32172
dtype: int64

In [13]:
data = data.drop_duplicates()
print(data.isnull().sum())

region        0
gender        0
age           0
height        0
weight        0
candj         0
snatch        0
deadlift      0
backsq        0
eat           0
background    0
experience    0
schedule      0
howlong       0
total_lift    0
dtype: int64


In [14]:
data.describe()

,age,height,weight,candj,snatch,deadlift,backsq,total_lift
count,32171.000000,3.217100e+04,32171.000000,32171.000000,32171.000000,3.217100e+04,3.217100e+04,3.217100e+04
mean,32.038979,3.293813e+02,176.872183,200.763296,152.998353,8.744933e+02,5.479909e+02,1.776246e+03
std,7.510277,4.676856e+04,33.550537,73.657778,73.820819,6.613757e+04,4.676746e+04,1.046007e+05
min,14.000000,0.000000e+00,5.000000,-45.000000,0.000000,-5.000000e+02,0.000000e+00,-2.200000e+01
25%,27.000000,6.600000e+01,155.000000,155.000000,115.000000,2.800000e+02,2.250000e+02,7.800000e+02
50%,31.000000,6.900000e+01,178.000000,205.000000,155.000000,3.650000e+02,2.950000e+02,1.025000e+03
75%,37.000000,7.200000e+01,196.000000,245.000000,190.000000,4.300000e+02,3.550000e+02,1.220000e+03
max,56.000000,8.388607e+06,1750.000000,5000.000000,8095.000000,8.388607e+06,8.388607e+06,1.678231e+07


## Dealing with Categorical Values

In [15]:
# One-Hot Encoding for categorical columns
categorical_cols = ['region', 'gender', 'eat', 'background', 'experience', 'schedule', 'howlong']
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [16]:
# Convert float columns to int
float_cols = ['age', 'height', 'weight', 'candj', 'snatch', 'deadlift', 'backsq', 'total_lift']
for col in float_cols:
    data[col] = data[col].astype(int)

In [17]:
# Verify the changes
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

data.dtypes

age                                                                                                                                                                                                                                                                                                                           int64
height                                                                                                                                                                                                                                                                                                                        int64
weight                                                                                                                                                                                                                                                                                                                        int64
candj                       

In [18]:
#Split data for data_v1
X_v1  = data.drop('total_lift', axis=1)
y_v1 = data['total_lift']
X_train_v1, X_test_v1, y_train_v1, y_test_v1 = train_test_split(X_v1, y_v1, test_size=0.2, random_state=42) 


## Save the raw data and push it using dvc tool

In [21]:
pip install dvc

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [22]:
!dvc init


Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In [23]:
!git status

On branch monika_v2
Your branch is ahead of 'origin/monika_v2' by 1 commit.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   .dvc/config

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .DS_Store
	deleted:    data/.gitignore
	deleted:    data/athletes.csv.dvc
	deleted:    data/athletes_v1.csv
	deleted:    data/data.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data.ipynb
	data/athletes.csv
	eda.ipynb



In [24]:
!git add .

In [25]:
!git commit -m "Initialize DVC"

[monika_v2 4fe2952] Initialize DVC
 9 files changed, 424453 insertions(+), 33731 deletions(-)
 create mode 100644 data.ipynb
 delete mode 100644 data/.gitignore
 create mode 100644 data/athletes.csv
 delete mode 100644 data/athletes.csv.dvc
 delete mode 100644 data/athletes_v1.csv
 delete mode 100644 data/data.ipynb
 create mode 100644 eda.ipynb


In [26]:
# data.to_csv('data_v1.csv', index=False)


In [27]:
# !dvc add data_v1.csv


⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding data_v1.csv to cache           0/? [00:00<?,     ?file/s]
  0%|          |Adding data_v1.csv to cache           0/1 [00:00<?,     ?file/s]
  0%|          |Adding data_v1.csv to cache           0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 13.68file/s]

To track the changes with git, run:

	g

## Set up remote storage with DVC

In [28]:
!dvc remote add -d myremote gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ


Setting 'myremote' as a default remote.


In [29]:
!dvc push


  0% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5| |0/1 [00:0
!
  0%|          |/Users/kajalshukla/Desktop/MLOPs/As0.00/? [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Desktop/MLOP0.00/19.9M [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Deskt8.00k/19.9M [00:03<2:09:18,    2.69kB/s]
  4%|▍         |/Users/kajalshukla/Desktop/768k/19.9M [00:03<00:57,     351kB/s]
 12%|█▏        |/Users/kajalshukla/Desktop2.47M/19.9M [00:03<00:13,    1.40MB/s]
 39%|███▉      |/Users/kajalshukla/Desktop7.73M/19.9M [00:03<00:02,    5.61MB/s]
100% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5|█|1/1 [00:0
1 file pushed                                                                   


In [30]:
!git add versions/data_v1.csv.dvc .dvc/config


In [31]:
!git commit -m "Track data_v1 with DVC"

[monika_v2 5697f93] Track data_v1 with DVC
 2 files changed, 9 insertions(+)
 create mode 100644 versions/data_v1.csv.dvc


# Version - 2

## Remove Outliers

In [36]:
# Remove outliers
data = data[data['weight'] < 1500]
# data = data[data['gender'] != '--']
data = data[data['age'] >= 18]
data = data[(data['height'] < 96) & (data['height'] > 48)]

data = data[
    (data['deadlift'] > 0) & (data['deadlift'] <= 1105) | 
    ((data['gender_Female'] == 1) & (data['deadlift'] <= 636))
]
data = data[(data['candj'] > 0) & (data['candj'] <= 395)]
data = data[(data['snatch'] > 0) & (data['snatch'] <= 496)]
data = data[(data['backsq'] > 0) & (data['backsq'] <= 1069)]

In [37]:
data.head(2)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|,eat_I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|,eat_I eat quality foods but don't measure the amount|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I eat strict Paleo|,eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat strict Paleo|I eat whatever is convenient|,eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|,eat_I eat whatever is convenient|Decline to answer|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|,eat_I weigh and measure my food|Decline to answer|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|,eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|,eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|,eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,eat_I weigh and measure my food|I eat whatever is convenient|,eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|,background_

In [39]:
data.dtypes

age                                                                                                                                                                                                                                                                                                                           int64
height                                                                                                                                                                                                                                                                                                                        int64
weight                                                                                                                                                                                                                                                                                                                        int64
candj                       

## Feature Engineering

### Body Mass Index

In [40]:
data['bmi'] = data['weight'] / (data['height'] / 100)**2

In [44]:
data = data.drop(columns=["eat_I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|",
                          "eat_I eat quality foods but don't measure the amount|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",
                          "eat_I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I eat strict Paleo|",
                          "eat_I eat strict Paleo|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|",
                          "eat_I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat strict Paleo|I eat whatever is convenient|",
                          "eat_I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat whatever is convenient|",
                          "eat_I eat whatever is convenient|Decline to answer|",
                          "eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|",
                          "eat_I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I weigh and measure my food|",
                          "eat_I weigh and measure my food|Decline to answer|",
                          "eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|",
                          "eat_I weigh and measure my food|I eat 1-3 full cheat meals per week|Decline to answer|",
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|",
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                         
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|",                                                                                                                                                                                                                
                          "eat_I weigh and measure my food|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                                            
                          "eat_I weigh and measure my food|I eat strict Paleo|",                                                                                                                                                                                                                                                                           
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                                                       
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|",                                                                                                                                                                                                                          
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                      
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat 1-3 full cheat meals per week|Decline to answer|",                                                                                                                                                                    
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                         
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat quality foods but don't measure the amount|I eat whatever is convenient|I eat 1-3 full cheat meals per week|Decline to answer|",                                                                                                                                       
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|",                                                                                                                                                                                                                                              
                          "eat_I weigh and measure my food|I eat strict Paleo|I eat whatever is convenient|I eat 1-3 full cheat meals per week|",                                                                                                                                                                                                          
                          "eat_I weigh and measure my food|I eat whatever is convenient|",                                                                                                                                                                                                                                                                 
                          "eat_I weigh and measure my food|I eat whatever is convenient|I eat 1-3 full cheat meals per week|"
                            ])

In [45]:
data.head(3)

,age,height,weight,candj,snatch,deadlift,backsq,total_lift,region_Asia,region_Australia,region_Canada East,region_Canada West,region_Central East,region_Europe,region_Latin America,region_Mid Atlantic,region_North Central,region_North East,region_North West,region_Northern California,region_South Central,region_South East,region_South West,region_Southern California,gender_Female,gender_Male,eat_I eat 1-3 full cheat meals per week|,background_I have no athletic background besides CrossFit|,background_I have no athletic background besides CrossFit|Decline to answer|,background_I have no athletic background besides CrossFit|I played college sports|,background_I have no athletic background besides CrossFit|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played professional sports|,background_I have no athletic background besides CrossFit|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|Decline to answer|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I played youth or high school level sports|I regularly play recreational sports|,background_I have no athletic background besides CrossFit|I regularly play recreational sports|,background_I played college sports|,background_I played college sports|Decline to answer|,background_I played college sports|I played professional sports|,background_I played college sports|I played professional sports|I regularly play recreational sports|,background_I played college sports|I regularly play recreational sports|,background_I played professional sports|,background_I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|,background_I played youth or high school level sports|Decline to answer|,background_I played youth or high school level sports|I played college sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|,background_I played youth or high school level sports|I played college sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played college sports|I regularly play recreational sports|,background_I played youth or high school level sports|I played professional sports|,background_I played youth or high school level sports|I played professional sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|,background_I played youth or high school level sports|I regularly play recreational sports|Decline to answer|,background_I regularly play recreational sports|,background_I regularly play recreational sports|Decline to answer|,experience_I began CrossFit by trying it alone (without a coac

## Save version 2

In [46]:
data.to_csv('versions/data_v2.csv', index=False)

In [47]:
!dvc add versions/data_v2.csv

⠋ Checking graph                                       core>
Adding...                                                                       
!
                                                                                
!
  0% Checking cache in '/Users/kajalshukla/Desktop/MLOPs/Assignments/Assignment-
                                                                                
!
  0%|          |Adding versions/data_v2.csv to cache  0/? [00:00<?,     ?file/s]
  0%|          |Adding versions/data_v2.csv to cache  0/1 [00:00<?,     ?file/s]
  0%|          |Adding versions/data_v2.csv to cache  0/1 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |Checking out /Users/kajalshukla/Deskto0/? [00:00<?,    ?files/s]
  0%|          |Checking out /Users/kajalshukla/Deskto0/1 [00:00<?,    ?files/s]
100% Adding...|████████████████████████████████████████|1/1 [00:00, 18.58file/s]

To track the changes with git, run:

	g

In [48]:
!dvc push versions/data_v2.csv


  0% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5| |0/1 [00:0
!
  0%|          |/Users/kajalshukla/Desktop/MLOPs/As0.00/? [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Desktop/MLOP0.00/17.3M [00:00<?,        ?B/s]
  0%|          |/Users/kajalshukla/Desktop8.00k/17.3M [00:01<54:48,    5.53kB/s]
  4%|▍         |/Users/kajalshukla/Desktop/704k/17.3M [00:01<00:27,     631kB/s]
  8%|▊         |/Users/kajalshukla/Desktop1.39M/17.3M [00:01<00:12,    1.37MB/s]
 39%|███▉      |/Users/kajalshukla/Desktop6.79M/17.3M [00:01<00:01,    8.90MB/s]
 65%|██████▌   |/Users/kajalshukla/Desktop11.3M/17.3M [00:01<00:00,    15.2MB/s]
100% Pushing to gdrive://1Zpt1VhpAx-XdTvY5W2Yhzs7-24SS23YQ/files/md5|█|1/1 [00:0
1 file pushed                                                                   


In [49]:
!git add data_v2.csv.dvc

fatal: pathspec 'data_v2.csv.dvc' did not match any files


In [51]:
!git add .

In [52]:
!git commit -m "Track data_v2 with DVC"

[monika_v2 7b3d1a6] Track data_v2 with DVC
 8 files changed, 32595 insertions(+), 61 deletions(-)
 create mode 100644 .gitignore
 create mode 100644 data/.DS_Store
 create mode 100644 versions/.gitignore
 create mode 100644 versions/data_v1.csv
 create mode 100644 versions/data_v2.csv.dvc
